In this notebook we transfer selected frames from the videos into a single HDF5 file.  
Before doing so, we flip them around both axes and resize them to 300x300.  
All these frames will be the input to our model, alongside the car distances we computed in a separate notebook.

In [1]:
import pandas as pd
from pandas.io.pytables import HDFStore
import numpy as np
import matplotlib.pyplot as plt
import cv2, os, sys, tables
sys.path.append('../scripts/')
import videoUtils

# Transfer frames to HDF5

The table with the file matching will help us with the left offset that we shall apply to the videos:

In [2]:
df_ord = pd.read_csv('../data/clean/files_matching.csv')
df_ord = df_ord[(~df_ord.dgps.isna()) & ~(df_ord.tesla_videos.isna() & df_ord.mercedes_videos.isna())]
df_ord.loc[:, [c for c in df_ord.columns if 'right' not in c]]

,dgps,tesla,mercedes,tesla_videos,mercedes_videos,tesla_left_offset,mercedes_left_offset
0,Thursday 1st manual.csv,2019-07-04-13-04-32.csv,2019-07-04-13-08-00.csv,V0350035.GLV,V0830086.GLV+V0830087.GLV,50.368,152.368
1,Thursday ACC max dist.csv,2019-07-04-13-04-32.csv,2019-07-04-13-44-50.csv,V0360037.GLV,V0840088.GLV,0.000,65.268
2,Thursday ACC min dist 60 90.csv,2019-07-04-14-15-08.csv,2019-07-04-14-15-16.csv,V0380039.GLV,V0850089.GLV,10.338,2.338
3,Thursday manual 50 100 inverse.csv,2019-07-04-15-34-19.csv,2019-07-04-15-34-58.csv,V0420043.GLV,V0880092.GLV,11.713,0.000
4,Thursday ACC min dist 50 100.csv,2019-07-04-14-50-23.csv,2019-07-04-14-50-58.csv,V0390040.GLV,V0860090.GLV,26.704,0.000
6,Thursday ACC min dist 50 100 with small perts.csv,2019-07-04-15-13-46.csv,2019-07-04-15-14-33.csv,V0400041.GLV,V0870091.GLV,17.746,0.000
9,Thursday ACC min dist 50 100 inverse 2.csv,NaN,2019-07-04-15-58-49.csv,NaN,V0890093.GLV,NaN,0.000


For convenience, we extract the (relevant) video names into a list:

In [3]:
videos = [video for video in df_ord.iloc[:, [3, 4]].values.reshape(-1) if not pd.isnull(video)]
videos

['V0350035.GLV',
 'V0830086.GLV+V0830087.GLV',
 'V0360037.GLV',
 'V0840088.GLV',
 'V0380039.GLV',
 'V0850089.GLV',
 'V0420043.GLV',
 'V0880092.GLV',
 'V0390040.GLV',
 'V0860090.GLV',
 'V0400041.GLV',
 'V0870091.GLV',
 'V0890093.GLV']

And process them, one by one. Notice that `V0830086.GLV+V0830087.GLV` is a special case.

In [4]:
out_path = '../data/clean/car_frames.h5'
with HDFStore('../data/clean/car_distances.h5', mode='r') as store_r:
    for video in videos:
        # Obtain dataframe with the distances
        if video == 'V0830086.GLV+V0830087.GLV':
            video = 'V0830086.GLV'
            df = store_r['V0830086_V0830087']
        else:
            df = store_r[video[:-4]]
        # Number of rows in the dataframe
        tot_shape = df.shape[0]
        # Number of non-na rows in the dataframe
        non_na = df[~df.iloc[:, 1].isna()].shape[0]
        # Obtain left offset of the video based on video name
        left_offset = df_ord[df_ord.tesla_videos.str.contains(video, na=False)].tesla_left_offset
        if left_offset.empty:
            left_offset = df_ord[df_ord.mercedes_videos.str.contains(video, na=False)].mercedes_left_offset
        # Extract from the pd.Series
        left_offset = left_offset.iloc[0]

        # Path of the video file (Tesla or Mercedes)
        path = '../data/raw/day 4/DCIM/100_VIRB/'
        if not os.path.exists(path + video):
            path = path.replace('4', '4 b')
        # Total number of frames in the video and fps
        tot_frames = videoUtils.get_total_frames(path + video)
        fps = videoUtils.get_fps(path + video)
        # Call function that transforms the video into frames with the required offset
        frames_ok, frame_count = videoUtils.video_2_frames(video, path, out_path, timestep=100, 
                                                           left_offset=left_offset*1000, max_frames=non_na)

        print('Video {}'.format(video))
        print('{} frames transformed, of a total of {}'.format(frames_ok, frame_count))
        print('The dataframe has {} non-nan values, and shape {}'.format(non_na, tot_shape))
        print('The left offset is {}'.format(left_offset))
        print('In the video, the total number of frames is {:0.6}'.format(tot_frames / fps / 0.1))
        print('Original number of frames is {:0.6}, with fps {:0.6}'.format(tot_frames, fps))
        print('--------------------------------------------')

    Processed 15000 out of 15791 frames, progress: 95.0%, elapsed: 1388 secc

Video V0350035.GLV
15791 frames transformed, of a total of 15791
The dataframe has 15791 non-nan values, and shape 18000
The left offset is 50.368
In the video, the total number of frames is 18008.0
Original number of frames is 53970.0, with fps 29.97
--------------------------------------------


    Processed 15000 out of 15791 frames, progress: 95.0%, elapsed: 1388 secc

Video V0830086.GLV
15791 frames transformed, of a total of 15791
The dataframe has 15791 non-nan values, and shape 21580
The left offset is 152.368
In the video, the total number of frames is 18008.0
Original number of frames is 53970.0, with fps 29.97
--------------------------------------------


    Processed 11000 out of 11720 frames, progress: 93.87%, elapsed: 1021 sec

...

Video V0400041.GLV
9162 frames transformed, of a total of 9163
The dataframe has 9163 non-nan values, and shape 9340
The left offset is 17.746
In the video, the total number of frames is 9338.66
Original number of frames is 27988.0, with fps 29.97
--------------------------------------------


    Processed 8000 out of 8975 frames, progress: 89.15%, elapsed: 743 sec

Video V0870091.GLV
8975 frames transformed, of a total of 8975
The dataframe has 8975 non-nan values, and shape 9800
The left offset is 0.0
In the video, the total number of frames is 9793.78
Original number of frames is 29352.0, with fps 29.97
--------------------------------------------


    Processed 4000 out of 4670 frames, progress: 85.67%, elapsed: 369 sec

Video V0890093.GLV
4670 frames transformed, of a total of 4670
The dataframe has 4670 non-nan values, and shape 4670
The left offset is 0.0
In the video, the total number of frames is 4677.34
Original number of frames is 14018.0, with fps 29.97
--------------------------------------------


# Check nodes in the HDF5

In [8]:
hdf5_file = tables.open_file(out_path, mode='r')
print([n for n in hdf5_file.walk_nodes()])
hdf5_file.close()

[/ (RootGroup) ''
  children := ['V0350035' (EArray), 'V0360037' (EArray), 'V0380039' (EArray), 'V0390040' (EArray), 'V0400041' (EArray), 'V0420043' (EArray), 'V0830086' (EArray), 'V0840088' (EArray), 'V0850089' (EArray), 'V0860090' (EArray), 'V0870091' (EArray), 'V0880092' (EArray), 'V0890093' (EArray)], /V0350035 (EArray(15791, 300, 300, 3), shuffle, blosc:lz4hc(9)) ''
  atom := UInt8Atom(shape=(), dflt=0)
  maindim := 0
  flavor := 'numpy'
  byteorder := 'irrelevant'
  chunkshape := (1, 300, 300, 3), /V0360037 (EArray(11720, 300, 300, 3), shuffle, blosc:lz4hc(9)) ''
  atom := UInt8Atom(shape=(), dflt=0)
  maindim := 0
  flavor := 'numpy'
  byteorder := 'irrelevant'
  chunkshape := (1, 300, 300, 3), /V0380039 (EArray(9076, 300, 300, 3), shuffle, blosc:lz4hc(9)) ''
  atom := UInt8Atom(shape=(), dflt=0)
  maindim := 0
  flavor := 'numpy'
  byteorder := 'irrelevant'
  chunkshape := (1, 300, 300, 3), /V0390040 (EArray(9457, 300, 300, 3), shuffle, blosc:lz4hc(9)) ''
  atom := UInt8Atom(sh